In [190]:
from __future__ import annotations
import re
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from typing import Optional, Hashable


In [185]:
def read_input(day: int) -> str:
    with open(f"./resources/day{day}.txt") as f:
        return f.read()

def read_input_lines(day: int) -> list[str]:
    with open(f"./resources/day{day}.txt") as f:
        return f.read().split("\n")

# Day 1 - Calorie Counting

In [183]:
input = read_input(1)
sums = [pd.Series(elve.split("\n")).astype(int).sum() for elve in input.split("\n\n")]
part1 = max(sums)
part2 = pd.Series(sums).sort_values(ascending=False).head(3).sum()
    
part1, part2

(67658, 200158)

# Day 2 - Rock Paper Scissors

In [184]:
input = read_input(2)

mapping = {"A": 1,"B": 2,"C": 3,"X": 1,"Y": 2,"Z": 3,}

def eval_points(opp: str, me: str):
    if opp == (me-1) or (opp == 3 and me==1): # win 1<2<3
        return me + 6
    elif opp == me: # draw
        return me + 3
    return me # loss

def eval_points2(opp:str, end:str):
    if end==1: # need to loose
        return opp-1 if opp!=1 else 3
    elif end==2: # need to draw
        return opp + 3
    return (opp+1 if opp!=3 else 1) + 6

df = pd.DataFrame([round.split(" ") for round in input.split("\n")], columns=["opp", "me"]).assign(opp=lambda df:df.opp.map(mapping), me=lambda df:df.me.map(mapping))
part1 = df.apply(lambda df:eval_points(df.opp, df.me), axis=1).sum()
part2 = df.apply(lambda df:eval_points2(df.opp, df.me), axis=1).sum()

part1, part2

(15337, 11696)

# Day 3 - Rucksack Reorganization

In [186]:
input = read_input_lines(3)

alph = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

def find_duplicate(left: str, right: str) -> int:
    for c in left:
        if c in right:
            return alph.index(c) + 1

def find_duplicate2(first: str, second: str, third: str) -> int:
    for c in first:
        if c in  second and c in third:
            return alph.index(c) + 1


df = pd.DataFrame([(line[:len(line) //2], line[len(line) // 2:]) for line in input], columns=["left", "right"])
part1 = df.apply(lambda df:find_duplicate(df.left, df.right), axis=1).sum()

ser = pd.Series([input[i:i+3] for i in range(0, len(input) -2, 3)])
part2 = ser.apply(lambda x: find_duplicate2(x[0], x[1], x[2])).sum()

part1, part2


(8085, 2515)

# Day 4 - Camp Cleanup

In [187]:
input = read_input_lines(4)

pattern = re.compile(r"(\d+)-(\d+),(\d+)-(\d+)")

def is_contained(a_start: int, a_end: int, b_start: int, b_end:int) -> int:
    if (a_start<=b_start and a_end>=b_end) or (b_start<=a_start and b_end>=a_end):
        return 1 # a,b fully contains c,d or c,d fully contains a,b'
    return 0

def overlap(a_start: int, a_end: int, b_start: int, b_end:int) -> int:
    if a_start > b_start:
        return overlap(b_start, b_end, a_start, a_end)
    elif a_end>=b_start:
        return 1
    return 0

df = pd.DataFrame([re.match(pattern, line).groups() for line in input], columns=["a_start", "a_end", "b_start", "b_end"]).astype(int)
part1 = df.apply(lambda df: is_contained(df.a_start, df.a_end, df.b_start, df.b_end), axis=1).sum()
part2 = df.apply(lambda df: overlap(df.a_start, df.a_end, df.b_start, df.b_end), axis=1).sum()

part1, part2

(550, 931)

# Day 5 - Supply Stacks

In [188]:
input = read_input_lines(5)

pattern = re.compile(r"move (\d+) from (\d+) to (\d+)")

def gen_stacks() -> list:
    return [ "SLW","JTNQ","SCHFJ","TRMWNGB", "TRLSDHQB", "MJBVFHRL", "DWRNJM", "BZTFHNDJ", "HLQNBFT"]

def get_result(stks: list) -> str:
    res = ""
    for stk in stks:
        res += stk[-1]
    return res

steps = [re.match(pattern, line).groups() for line in input]

def part1():
    stks = gen_stacks()
    for step in steps:
        amount = int(step[0])
        for _ in range(amount):
            crate = stks[int(step[1]) -1][-1]
            stks[int(step[1]) -1] = stks[int(step[1]) -1].removesuffix(crate)
            stks[int(step[2]) -1] += crate
    return get_result(stks)

def part2():
    stks = gen_stacks()
    for step in steps:
        amount = int(step[0])
        crates = stks[int(step[1]) -1][-amount:]
        stks[int(step[1]) -1] = stks[int(step[1]) -1].removesuffix(crates)
        stks[int(step[2]) -1] += crates
    return get_result(stks)

part1(), part2()

('RLFNRTNFB', 'MHQTLJRLB')

# Day 6 - Tuning Trouble

In [189]:
input = read_input(6)

def get_marker(input: str, packet_size: int):
    for i in range(len(input) - packet_size):
        current_pack = input[i:i+packet_size]
        if len(current_pack) == len(set(current_pack)):
            return i + packet_size
            
part1 = get_marker(input, 4)
part2 = get_marker(input, 14)

part1, part2

(1175, 3217)

## Day 7 - No Space Left on Device

In [191]:
input = read_input_lines(7)

CD_PATTERN = re.compile(r"\$ cd (\S+)")
LS_FILE_PATTERN = re.compile(r"(\d+) (.+)")
LS_DIR_PATTERN = re.compile(r"dir (.+)")

CD_UP_CMD = "$ cd .."
CD_ROOT_CMD = "$ cd /"

@dataclass(eq=True)
class Item(Hashable):
    name: str = field(compare=True)
    def __hash__(self) -> int:
        return hash(self.name)

@dataclass(eq=True)
class File(Item):
    __hash__ = Item.__hash__
    size: int

@dataclass(eq=True)
class Dir(Item):
    __hash__ = Item.__hash__
    dirs: set[Dir] = field(default_factory=set)
    files: set[File] = field(default_factory=set)
    
    def cd(self, name: str) -> Optional[Dir]:
        for dir in self.dirs:
            if dir.name == name:
                return dir
        return None
    
    @property
    def file_size(self):
        sum = 0
        for file in self.files:
            sum += file.size
        return sum 

    def total_size(self, func = None, *args) -> int:
        sum = 0
        for dir in self.dirs:
            dir_size = dir.total_size(func, *args)
            if func is not None:
                func(dir_size, *args)
            sum+=dir_size
        return sum + self.file_size

def size_less_than_threshold(dir_size : int, result: list[int], threshold: int):
    if dir_size<=threshold:
        result.append(dir_size)

def size_higher_than_threshold(dir_size : int, result: list[int], threshold: int):
    if dir_size>=threshold:
        result.append(dir_size)
    
def get_filesystem(input: list[str]) -> Dir:
    root = Dir(name="/")
    stack = [root]
    for line in input:
        current = stack[-1]
        if line == CD_UP_CMD:
            stack.pop()
        elif line == CD_ROOT_CMD:
            stack = [root]
        elif re.match(CD_PATTERN, line) is not None: # cd command
            name = re.match(CD_PATTERN, line).groups()[0]
            stack.append(current.cd(name))
        elif re.match(LS_FILE_PATTERN, line) is not None: # ls found file
            match = re.match(LS_FILE_PATTERN, line).groups()
            file = File(name=match[1], size=int(match[0]))
            current.files.add(file)
        elif re.match(LS_DIR_PATTERN, line) is not None:
            match = re.match(LS_DIR_PATTERN, line).groups()
            dir = Dir(name = match[0])
            current.dirs.add(dir)
    return root

def part1(input: list[str]):
    root = get_filesystem(input)
    values = []
    root.total_size(size_less_than_threshold, values, 100000)
    return sum(values)

def part2(input: list[str]):
    root = get_filesystem(input)
    actual_size = root.total_size()
    space_to_clear = actual_size - 40000000
    values = []
    root.total_size(size_higher_than_threshold, values, space_to_clear)
    return min(values)

part1 = part1(input)
part2 = part2(input)

part1, part2

(1315285, 9847279)

# Day 8 - Treetop Tree House

In [192]:

input = read_input_lines(8)
matrix = np.matrix([list(row) for row in input], dtype=int)

def get_number_visible_trees(matrix: np.matrix) -> int:
    transposed = matrix.T
    res = 0
    for i in range(1, len(matrix)-1): # iterate inner rows, i index of row
        row = np.squeeze(np.asarray(matrix[i]))
        for j in range(1, len(row)-1): # iterate inner columns, j index of column
            col = np.squeeze(np.asarray(transposed[j]))
            if min(max(row[:j]),max(row[j+1:]),max(col[:i]),max(col[i+1:])) < row[j]:
                res+=1
    return res+ 2*matrix.shape[0] + 2*matrix.shape[1] -4

def get_vd(height: int, arr: np.ndarray, flip: bool = False):
    if flip:
        arr = arr[::-1]
    for i, value in enumerate(arr):
        if height <= value:
            return i+1
    return len(arr)

def get_scenic_score(matrix: np.matrix) -> int:
    transposed = matrix.T
    result = 0
    for i in range(1, len(matrix) -1):
        row = np.squeeze(np.asarray(matrix[i]))
        for j in range(1, len(row) -1):
            col = np.squeeze(np.asarray(transposed[j]))
            cur = row[j]
            prod = get_vd(cur, row[:j], flip=True) *  get_vd(cur, row[j+1:]) * get_vd(cur, col[:i], flip=True) *  get_vd(cur, col[i+1:])
            if prod > result:
                result = prod
    return result

part1 = get_number_visible_trees(matrix)
part2 = get_scenic_score(matrix)

part1, part2

(1543, 595080)

# Day 9 - Rope Bridge

In [193]:

input = read_input_lines(9)
steps = [line.split(" ") for line in input]
steps = [(step[0], int(step[1])) for step in steps]

signum = lambda x: 1 if x>0 else -1 if x<0 else 0
moving_distance = lambda x: (abs(x)-1)*signum(x)

@dataclass
class Knot:
    x: int
    y: int
    head: Optional[Knot] = None

    @property
    def pos(self):
        return (self.x, self.y)
    
    def update(self):
        dx = self.head.x - self.x
        dy = self.head.y - self.y
        if dx == dy == 0: # same pos
            return
        if abs(dx) == abs(dy) == 1: # direct diagonal
            return
        if dx == 0: # both on same x-axis
            self.y += moving_distance(dy)
        elif dy == 0: # both on same y-axis
            self.x += moving_distance(dx)
        else: # two-wide diagonal
            self.x += signum(dx)
            self.y += signum(dy)

    def move(self, direction: str):
        if direction == "L":
            self.x -= 1
        elif direction == "R":
            self.x += 1
        elif direction == "U":
            self.y += 1
        elif direction == "D":
            self.y -= 1

def gen_rope(length: int) -> tuple[Knot, list[Knot], Knot]:
    head = Knot(0,0)
    current_head = head
    l = [head]
    for _ in range(length-1):
        knot = Knot(0,0,current_head)
        current_head = knot
        l.append(knot)
    return head, l, l[-1]

def get_visited_positions(rope_length: int, steps: list[tuple[str, int]]):
    head, rope, tail = gen_rope(rope_length)
    visits = set()
    visits.add(tail.pos)
    for step in steps:
        direction = step[0]
        length = step[1]
        for _ in range(length):
            head.move(direction)
            for knot in rope[1:]:
                knot.update()
            visits.add(tail.pos)
    return len(visits)

part1 = get_visited_positions(2, steps)
part2 = get_visited_positions(10, steps)

part1, part2

(6236, 2449)

# Day 10 - Cothode-Ray Tube

In [292]:
input = read_input_lines(10)

NOOP = "noop"
DEFAULT_SCREEN_WIDTH = 40
DEFAULT_SCREEN_HEIGHT = 6

def gen_screen(height: int = DEFAULT_SCREEN_HEIGHT) -> list[str]:
    return ["" for _ in range(height)]

def render_screen(screen: list[str]):
    string = ""
    for line in screen:
        string += line + "\n"
    return string

def draw_screen(x: int, cycle: int, screen: list[str]):
    sprite = [x-1, x, x+1]
    crt_pointer = ((cycle-1)%40)
    row = (cycle-1) // 40
    if crt_pointer in sprite:
        screen[row] += "#"
    else:
        screen[row] += "."

def get_signal_strength(x: int, cycle: int, results: list[int]):
    if (cycle+20)%DEFAULT_SCREEN_WIDTH == 0:
        results.append(x*cycle)

def run(input: list[str], results: list, func):
    cycle = 1
    x = 1
    addx_ready = False
    index = 0
    while index < len(input):
        addx = 0
        line = input[index]
        if line == NOOP:
            index += 1
        elif addx_ready:
            addx = addx = int(line.split()[1])
            addx_ready = False
            index += 1
        else:
            addx_ready = True
        func(x, cycle, results)
        cycle += 1
        x+= addx
    return results

part1 = sum(run(input, [], get_signal_strength))
part2 = run(input, gen_screen(), draw_screen)

print(part1)
print(render_screen(part2))

14920
###..#..#..##...##...##..###..#..#.####.
#..#.#..#.#..#.#..#.#..#.#..#.#..#....#.
###..#..#.#....#..#.#....###..#..#...#..
#..#.#..#.#....####.#....#..#.#..#..#...
#..#.#..#.#..#.#..#.#..#.#..#.#..#.#....
###...##...##..#..#..##..###...##..####.

